In [19]:
%%time

import csv

def makeHeadfile(rawFileName='FlightDelays.csv', newFileName='FlightDelays_heads.csv', headSize=500, merge=False):
    data = []
    with open(rawFileName) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            data.append(row)
            line_count += 1
            if line_count > headSize:
                break
    with open(newFileName, mode='w', newline='') as head_file:
        head_writer = csv.writer(head_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        head_writer.writerows(data)
    
    return data[0]

#save head of FlightDelays_1k_sample.csv file
raw_col = makeHeadfile(rawFileName='FlightDelays_1k_sample.csv',headSize=1000)

Wall time: 15 ms


In [14]:
%%time

def dataLoad(baseFileName, addingFileName):
    '''
    data is a list of FlightDelays_head file.
    merge is a list of lists of AirFares file divided by its year and quaters to decrease search iteration.
    '''
    data = []
    merge=[]
    
    with open(baseFileName) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            data.append(row)
            
    with open(addingFileName) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            qt = []
            init = 0
            for row in csv_reader:
                if row[0] == '2018' and row[1] == '1':
                    qt.append(row)
                elif row[0] == '2018' and row[1] == '2':
                    if init==0:
                        merge.append(qt)
                        qt=[]
                        init = 1
                    qt.append(row)
                elif row[0] == '2018' and row[1] == '3':
                    if init==0:
                        merge.append(qt)
                        qt=[]
                        init = 1
                    qt.append(row)
                elif row[0] == '2018' and row[1] == '4':
                    if init==0:
                        merge.append(qt)
                        qt=[]
                        init = 1
                    qt.append(row)
                elif row[0] == '2019' and row[1] == '1':
                    if init==0:
                        merge.append(qt)
                        qt=[]
                        init = 1
                    qt.append(row)
                elif row[0] == '2019' and row[1] == '2':
                    if init==0:
                        merge.append(qt)
                        qt=[]
                        init = 1
                    qt.append(row)
                elif row[0] == '2019' and row[1] == '3':
                    if init==0:
                        merge.append(qt)
                        qt=[]
                        init = 1
                    qt.append(row)
                else:
                    AFindex = row
            merge.append(qt)
    return data, merge, AFindex


def addCol(data, colNames):
    for newcol in colNames:
        data = [row + [''] for row in data]
        data[0][-1] = newcol
    return data


def mergeFlightDelaysAndAirFares(data, merge):
    newData = []
    for data_row in data:
        if data_row[0] == '2018' and data_row[1] == '1':
            for i, merge_row in enumerate(merge[0]):
                if data_row[9] == merge_row[2] and data_row[10] == merge_row[4]:
                    data_row[-8:] = merge[0][i][-8:]
        elif data_row[0] == '2018' and data_row[1] == '2':
            for i, merge_row in enumerate(merge[1]):
                if data_row[9] == merge_row[2] and data_row[10] == merge_row[4]:
                    data_row[-8:] = merge[0][i][-8:]
        elif data_row[0] == '2018' and data_row[1] == '3':
            for i, merge_row in enumerate(merge[2]):
                if data_row[9] == merge_row[2] and data_row[10] == merge_row[4]:
                    data_row[-8:] = merge[0][i][-8:]
        elif data_row[0] == '2018' and data_row[1] == '4':
            for i, merge_row in enumerate(merge[3]):
                if data_row[9] == merge_row[2] and data_row[10] == merge_row[4]:
                    data_row[-8:] = merge[0][i][-8:]
        elif data_row[0] == '2019' and data_row[1] == '1':
            for i, merge_row in enumerate(merge[4]):
                if data_row[9] == merge_row[2] and data_row[10] == merge_row[4]:
                    data_row[-8:] = merge[0][i][-8:]            
        elif data_row[0] == '2019' and data_row[1] == '2':
            for i, merge_row in enumerate(merge[5]):
                if data_row[9] == merge_row[2] and data_row[10] == merge_row[4]:
                    data_row[-8:] = merge[0][i][-8:]
        elif data_row[0] == '2019' and data_row[1] == '3':
            for i, merge_row in enumerate(merge[6]):
                if data_row[9] == merge_row[2] and data_row[10] == merge_row[4]:
                    data_row[-8:] = merge[0][i][-8:]
        
            
        
        newData.append(data_row)
    return newData

def removeOtherNoise(data, noise_list):
    rm = []
    ndata = []
    for noise in noise_list:
        for i, att in enumerate(mergedData[0]):
            if att == noise:
                break
        for line in mergedData:
            line.pop(i)
            
    return mergedData

Wall time: 0 ns


In [17]:
%%time
baseFileName = 'FlightDelays_heads.csv'
addingFileName = 'AirFares.csv'

#Load csv files of FlightDelays_heads and AirFares
data, merge, AFindex = dataLoad(baseFileName, addingFileName)


#make new columns to FlightDelays_heads from AirFares
newCols = AFindex[6:]
data = addCol(data, newCols)

#Merge fare information from AirFares to FlightDelays_head file by searching their airport information.
mergedData = mergeFlightDelaysAndAirFares(data, merge)

noiseList = ['ARR_TIME','ARR_DELAY_NEW', 'ARR_DEL15', 'ARR_DELAY_GROUP', 'ARR_TIME_BLK','WHEELS_OFF','WHEELS_ON']

regularizedData = removeOtherNoise(mergedData, noiseList)
print(regularizedData[0])

['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE', 'CARRIER', 'FL_NUM', 'Route', 'ORIGIN', 'DEST', 'DEST_CITY', 'DEST_STATE', 'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'DEP_DELAY_NEW', 'DEP_DEL15', 'DEP_DELAY_GROUP', 'DEP_TIME_BLK', 'TAXI_OUT', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_DELAY', 'CANCELED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'PASSENGERS', 'EMPFULL', 'EMPPART', 'EMPTOTAL', 'EMPFTE', 'NET_INCOME', 'OP_REVENUES', 'nsmiles', 'fare', 'carrier_lg', 'large_ms', 'fare_lg', 'carrier_low', 'lf_ms', 'fare_low']
Wall time: 2.3 s


In [18]:
import pandas as pd

df = pd.DataFrame.from_records(regularizedData)
df

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,CARRIER,FL_NUM,Route,ORIGIN,...,NET_INCOME,OP_REVENUES,nsmiles,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low
1,2018,1,1,1,1,2018-01-01,9E,3331,42,ABY,...,-126.37,674.25,,,,,,,,
2,2018,1,1,1,1,2018-01-01,9E,3940,42,ABY,...,-126.37,674.25,,,,,,,,
3,2018,1,1,2,2,2018-01-02,9E,3409,42,ABY,...,-126.37,674.25,,,,,,,,
4,2018,1,1,2,2,2018-01-02,9E,3940,42,ABY,...,-126.37,674.25,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,2018,1,1,22,1,2018-01-22,9E,3302,203,ATL,...,-126.37,674.25,331,215.86,DL,0.9638,212.3,DL,0.9638,212.3
997,2018,1,1,23,2,2018-01-23,9E,3315,203,ATL,...,-126.37,674.25,331,215.86,DL,0.9638,212.3,DL,0.9638,212.3
998,2018,1,1,23,2,2018-01-23,9E,3373,203,ATL,...,-126.37,674.25,331,215.86,DL,0.9638,212.3,DL,0.9638,212.3
999,2018,1,1,23,2,2018-01-23,9E,3334,203,ATL,...,-126.37,674.25,331,215.86,DL,0.9638,212.3,DL,0.9638,212.3


Note
------
LATE_AIRCRAFT_DELAY and WHEELS ON and OFF are important due to runway capacity also wheels on and off
Relative time difference of CRS_DEP_TIME, DEP_TIME, DEP_DELAY, DEP_TIME, TAXI_OUT, WHEELS_ON, WHEELS_OFF .... timestamps to find a pattern of their relationships on each routes and season quarters.
Based on Air Travel Time in the paper, set CRS departure time to 0, and change following time related to CRS departure time.
However, we also need to consider it's absoulte time.

FlightDate !~ ARRDelay

fill None to 0
need to remove canceled flights
need to remove route name and airpot name since we have routh numbers and distance...